**Задание 1**  
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

Наименование вакансии.
Предлагаемую зарплату (отдельно минимальную и максимальную).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
import json
import requests
from pprint import pprint
from bs4 import BeautifulSoup as bs
import pandas as pd

main_link = 'https://hh.ru'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}

In [2]:
# Сбор информации о вакансиях с площадки Head Hanter

vacancies = []

num_page = int(input('Введите количество страниц для поиска: '))
search_text = input('Введите ключевое слово для поиска: ')

for i in range(num_page):
    params = {
        'clusters': 'true',  
        'enable_snippets': 'true',  
        'text': search_text,
        'L_save_area': 'true',
        'area': '1', # Поиск проходит по вакансиям 
        'from': 'cluster_area',
        'showClusters': 'false',
        'page':i
    }

    html = requests.get(main_link + '/search/vacancy', params=params, headers=headers)
    soup = bs(html.text,'html.parser')

    vacancies_block = soup.find('div',{'class':'vacancy-serp'})
    vacancies_list = vacancies_block.find_all('div',{'class':'vacancy-serp-item'}) 
    
    for vacancy in vacancies_list:
        vacancy_data = {}
        vacancy_name = vacancy.find('a', {'class':['bloko-link', 'HH-LinkModifier']}).getText()
        vacancy_link = vacancy.find('a', {'class':['bloko-link', 'HH-LinkModifier']})['href']
        employer_link = main_link + vacancy.find('div', {'class':['vacancy-serp-item__meta-info']}).find('a')['href']
        
        vacancy_data['vacancy_name'] = vacancy_name
        vacancy_data['vacancy_link'] = vacancy_link
        vacancy_data['employer_link'] = employer_link

        salalary_data = vacancy.find('div', {'class':['vacancy-serp-item__sidebar']}).getText()
        salary_list = salalary_data.replace(u'\xa0', '').replace('-', ' ').split()

        if 'от' in salary_list:
            vacancy_data['salary_min'] = salary_list[1]
            vacancy_data['salary_max'] = None
            vacancy_data['salary_unit'] = salary_list.pop()
        elif 'до' in salary_list:
            vacancy_data['salary_min'] = None
            vacancy_data['salary_max'] = salary_list[1]
            vacancy_data['salary_unit'] = salary_list.pop()
        elif len(salary_list) > 2:
            vacancy_data['salary_min'] = salary_list[0]
            vacancy_data['salary_max'] = salary_list[1]
            vacancy_data['salary_unit'] = salary_list.pop()
        else:
            vacancy_data['salary_min'] = None
            vacancy_data['salary_max'] = None
            vacancy_data['salary_unit'] = None

        vacancies.append(vacancy_data)

print(f'По вашему запросу найдено {len(vacancies)} вакансий')

Введите количество страниц для поиска: 40
Введите ключевое слово для поиска: python
По вашему запросу найдено 2000 вакансий


In [3]:
result = pd.DataFrame(vacancies)
result.head()

,employer_link,salary_max,salary_min,salary_unit,vacancy_link,vacancy_name
0,https://hh.ru/employer/1776381,180000,100000,руб.,https://hh.ru/vacancy/37674249?query=python,Программист-разработчик (python)
1,https://hh.ru/employer/1455?dpt=hh-1455-ds,None,None,None,https://hh.ru/vacancy/37844601?query=python,Machine Learning Developer
2,https://hh.ru/employer/2279941,140000,80000,руб.,https://hh.ru/vacancy/38814959?query=python,Junior Python developer
3,https://hh.ru/employer/2386651,120000,100000,руб.,https://hh.ru/vacancy/39085104?query=python,Python разработчик
4,https://hh.ru/employer/4835563,170000,120000,руб.,https://hh.ru/vacancy/38660476?query=python,Программист Python


In [4]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 6 columns):
employer_link    2000 non-null object
salary_max       305 non-null object
salary_min       376 non-null object
salary_unit      468 non-null object
vacancy_link     2000 non-null object
vacancy_name     2000 non-null object
dtypes: object(6)
memory usage: 93.8+ KB
